In [15]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Lambda, Dense, Input
from keras.callbacks import ModelCheckpoint 

from keras import optimizers
from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files 
from tqdm import tqdm


# Load and examine the dataset

In [2]:
def load_dataset(path):
    # the sklearn.datasets.load_files function takes a dataset directory as an input
    # for each folder within the directory, the folder name should be a category
    # within each folder will be samples corresponding to their category
    # load_files returns a bunch object which is a dictionary with 'DESC', 
    # 'data', 'filenames', 'target' as keys
    data = load_files(path)
    # an array of file paths or directories
    data_files = np.array(data['filenames']) 
    data_labels = np_utils.to_categorical(np.array(data['target']), 10)
    return data_files, data_labels

In [ ]:
train_data_dir = "imgs/train"
data_directory = train_data_dir

In [3]:
train_files, train_labels = load_dataset(data_directory)
print("There are {} total training images.".format(len(train_files)))

There are 20 total training images.


# Model building

I want to use transfer learning to boost the performance, and to obtain a proper weighted top layer, I first have to get the bottleneck features from my own dataset and then train the top layer with bottleneck features as inputs

### About preprocess_input

Getting the 4D tensor ready for VGG19, and for any other pre-trained model in Keras, requires some additional processing. For example, for VGG19, first, the RGB image is converted to BGR by reordering the channels.  All pre-trained models have the additional normalization step that the mean pixel (expressed in RGB as $[103.939, 116.779, 123.68]$ and calculated from all pixels in all images in ImageNet) must be subtracted from every pixel in each image.  This is implemented in the imported function `preprocess_input`.

See below link for more detailed operations: [preprocess_input](https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py)

# Hyperparameters setting

In [7]:
# I intentionally set the `test_size` as `None` here so you can modify the proportion in the `model.fit()` or other places
X_train, X_valid, y_train, y_valid = train_test_split(train_files, train_labels, test_size=None)

In [8]:
y_train

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]], dtype=float32)

In [9]:
# Helper functions to process the training and validation data
def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=target_size)
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(path) for path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

## Model specific settings starts here

# NOTICE:

As the preprocess_input varies a bit for different models,to ensure we did not miss any subtle difference, I copied the data to re-process the data at the expense of memory

In [ ]:
INPUT_DATA = paths_to_tensor(X_train)

### Resnet 50

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input

resnet_train_data = preprocess_input(np.copy(INPUT_DATA))

In [ ]:
ResNet50_model = ResNet50(include_top=False, weights='imagenet')

bottleneck_features_training = ResNet50_model.predict(resnet_train_data)

np.savez('bottleneck_features_train_ResNet50.npz', bottleneck_features_training)

### InceptionV3

In [ ]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input

inception_v3_train_data = preprocess_input(np.copy(INPUT_DATA))

In [11]:
InceptionV3_model = InceptionV3(include_top=False, weights='imagenet')

bottleneck_features_training = InceptionV3_model.predict(inception_v3_train_data)

np.savez('bottleneck_features_train_InceptionV3.npz', bottleneck_features_training)

### Xception

In [ ]:
from keras.applications.xception import Xception, preprocess_input

xception_train_data = preprocess_input(np.copy(INPUT_DATA))

In [ ]:
Xception_model = Xception(include_top=False, weights='imagenet')

bottleneck_features_training = Xception_model.predict(xception_train_data)

np.savez('bottleneck_features_train_Xception.npz', bottleneck_features_training)